# Get Data

Note: data is the three scale channel features

In [22]:
import gc
import h5py
import numpy as np

load_path='F:/wangqi/nistcomparison/AAPlantD1_2GHz_TX1_hpol_run4_pp.mat'
load_mat=h5py.File(load_path,'r')
IQdata=load_mat['IQdata'][3600:39600,0:256].view('complex')
locations=120
records=300
# IQdata=load_mat['IQdata'][3600:21600,0:256].view('complex')
# locations=60

# Hyper-parameters

In [23]:
# training parameters
seed = 0                        # random seed
batch_size = 32                # batch size
num_epoch = 50                   # the number of training epoch
patience = 5                  # If no improvement in 'patience' epochs, early stop
learning_rate = 0.0005          # learning rate
weight_decay=1e-5
model_path = './model.ckpt'     # the path where the checkpoint will be saved

# model parameters
k=1  #H为k+1个h
nf=1 #参考CSI的数量
index=[0,1]
# index=[0,3,1,4,2,5]
# index=[0,1,2,3,4,5]
d=2  #eve在alice后面d-1个位置处尾随
input_channels=4
len_sequence = 256                  # the input dim of the model, you should not change the value
output_dim=2
mu=0.4
std=0.1

# creat a image

In [24]:
import numpy as np
from numpy import fft
import torchvision.transforms as transforms
def convert_pic(h):
    width, height=h.shape
    img=np.zeros([input_channels,width,height]).astype(np.float32)
    img[0,:,:]=np.abs(h)
    img[1,:,:]=np.angle(h)
    img[2,:,:]=np.abs(fft.fft2(h))
    img[3,:,:]=np.angle(fft.fft2(h))
    return img 



# creatH

In [25]:
def creatHimag_Alice(idx, k, nf):
    H=np.zeros((k+nf,len_sequence)).astype(complex)
    H=IQdata[idx-nf:idx+k]
    # print(np.shape(H))
    return  np.transpose(H[index])
def creatHimag_Eve(idx, k, nf, d):
    H=np.zeros((k+nf,len_sequence)).astype(complex)
    H[nf:k+nf,:]=IQdata[idx-d*records:idx+k-d*records]
    H[0:nf,:]=IQdata[idx-nf:idx]
    return  np.transpose(H[index])

# Preprocess data 

In [26]:
recs=records*d
num_dataall=locations*records-recs-10
dataall_HimagA=np.zeros((num_dataall,len_sequence, k+nf)).astype(complex)
dataall_HimagE=np.zeros((num_dataall,len_sequence, k+nf)).astype(complex)
for i in range(recs,locations*records-10):
    dataall_HimagA[i-recs,:,:]=creatHimag_Alice(i,k=k,nf=nf)
    dataall_HimagE[i-recs,:,:]=creatHimag_Eve(i,k=k,nf=nf,d=d)

dataall_TW=np.concatenate((dataall_HimagA,dataall_HimagE),axis=0)
location=[i for i in range(2)]
test_data=dataall_TW
test_lab=[val for val in location for i in range(int(num_dataall))]
train_data=dataall_TW[0:int(num_dataall*2):2]
train_label=[val for val in location for i in range(int(num_dataall/2))]
valid_data=dataall_TW[1:int(num_dataall*2):2]
valid_label=[val for val in location for i in range(int(num_dataall/2))]

del dataall_HimagA, dataall_HimagE, dataall_TW
gc.collect()

187

# Define Dataset

In [27]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class hDataset(Dataset):
    def __init__(self, k, X, y=None):
        super(hDataset).__init__()
        self.data = X
        self.k = k
        if y is not None:
            self.label = torch.LongTensor(y)
        else:
            self.label = None

    def __getitem__(self, idx):
        h = self.data[idx]
        h = convert_pic(h)
        if self.label is not None:
            return h, self.label[idx]
        else:
            return h

    def __len__(self):
        return len(self.data)


# Prepare dataset and model

In [28]:
import torch
import torch.nn as nn
import torch.optim as optim
from model import resnet34
import os
# get dataset
train_set = hDataset(k,train_data, train_label)
valid_set = hDataset(k,valid_data, valid_label)

# remove raw feature to save memory
del train_data, valid_data, valid_label
gc.collect()

# get dataloader
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=False)

device = 'cuda:1' if torch.cuda.is_available() else 'cpu'
print(f'DEVICE: {device}')
# create model, define a loss function, and optimizer
net = resnet34()
in_channel = net.fc.in_features
net.fc = nn.Linear(in_channel, output_dim)
net.to(device)

criterion = nn.CrossEntropyLoss() 
# criterion = nn.BCELoss() 
# optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)
params = [p for p in net.parameters() if p.requires_grad]
optimizer = optim.Adam(params, lr=learning_rate, weight_decay= weight_decay)

DEVICE: cuda:1


# Training

In [29]:
import tqdm
import sys
stale=0
best_acc = 0.0
_exp_name = "HM_hrun4_resnettwi_sample"
train_losses=[]
train_accss=[]
valid_losses=[]
valid_accss=[]

for epoch in range(num_epoch):
    net.train()
    train_loss=[]
    train_accs=[]
# ---training---
    # train_bar = tqdm(train_loader, file=sys.stdout)
    # for batch in enumerate(train_bar):
    for batch in train_loader:    
        H, labels =batch
        H = H.to(device)
        labels = labels.to(device)
   
        optimizer.zero_grad()
        logits = net(H.to(device))
        loss =criterion(logits, labels.to(device))
        loss.backward()
        # grad_norm=nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)
        optimizer.step()
        
        acc=(logits.argmax(dim=-1) == labels.to(device)).float().mean()
        
        # Record the loss and accuracy.
        train_loss.append(loss.item())
        train_accs.append(acc)
        
    train_loss = sum(train_loss) / len(train_loss)
    train_acc = sum(train_accs) / len(train_accs)
    train_losses.append(train_loss)
    train_accss.append(np.array(train_acc.cpu()))
    # Print the information.
    print(f"[ Train | {epoch + 1:03d}/{num_epoch:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")

# ---validation---
    net.eval()
    valid_loss=[]
    valid_accs=[]

    # for batch in tqdm(valid_loader):
    for batch in valid_loader:
        # A batch consists of image data and corresponding labels.
        H, labels = batch
        #imgs = imgs.half()

        # We don't need gradient in validation.
        # Using torch.no_grad() accelerates the forward process.
        with torch.no_grad():
            logits = net(H.to(device))

        # We can still compute the loss (but not the gradient).
        loss = criterion(logits, labels.to(device))

        # Compute the accuracy for current batch.
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

        # Record the loss and accuracy.
        valid_loss.append(loss.item())
        valid_accs.append(acc)
        #break

    # The average loss and accuracy for entire validation set is the average of the recorded values.
    valid_loss = sum(valid_loss) / len(valid_loss)
    valid_acc = sum(valid_accs) / len(valid_accs)
    valid_losses.append(valid_loss)
    valid_accss.append(np.array(valid_acc.cpu()))
    # Print the information.
    print(f"[ Valid | {epoch + 1:03d}/{num_epoch:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")        

    # update logs
    if valid_acc > best_acc:
        with open(f"./{_exp_name}_log.txt","a"):
            print(f"[ Valid | {epoch + 1:03d}/{num_epoch:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f} -> best")
    else:
        with open(f"./{_exp_name}_log.txt","a"):
            print(f"[ Valid | {epoch + 1:03d}/{num_epoch:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")


    # save models
    # if epoch % 50 == 0:
    if valid_acc > best_acc:
        print(f"Best model found at epoch {epoch}, saving model")
        torch.save(net.state_dict(), f"{_exp_name}_best.ckpt") # only save best to prevent output memory exceed error
        best_acc = valid_acc
        stale = 0
    else:
        stale += 1
        if stale > patience:
            print(f"No improvment {patience} consecutive epochs, early stopping")
            break   

KeyboardInterrupt: 

# Loss and Acc show

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import style

plt.figure(1)
plt.plot(train_losses, linestyle="--", label="train")
plt.plot(valid_losses, linestyle="-", label="valid")
plt.legend(loc="best")
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show

plt.figure(2)
plt.plot(train_accss, linestyle="--", label="train")
plt.plot(valid_accss, linestyle="-", label="valid")
plt.legend(loc="best")
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.show

# Testing

In [ ]:
import numpy as np
test_set = hDataset(k, test_data, test_lab,tfm=test_tfm)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)

model_best = resnet34(num_classes=output_dim).to(device)
model_best.load_state_dict(torch.load(f"{_exp_name}_best.ckpt"))
model_best.eval()
prediction = []
with torch.no_grad():
    for data,_ in test_loader:
        test_pred = model_best(data.to(device))
        test_label = np.argmax(test_pred.cpu().data.numpy(), axis=1)
        prediction += test_label.squeeze().tolist()

accuracy=1 - np.count_nonzero(np.array(prediction)-np.array(test_lab))/len(prediction)

print(accuracy)